# Washington State DOT Traffic API

source: [https://wsdot.wa.gov/traffic/api](https://wsdot.wa.gov/traffic/api)


In [20]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv
from pathlib import Path
from sqlalchemy import create_engine

## Import env variables


In [21]:
# load the .env file
load_dotenv()

# API variables
api_key = os.getenv("API_KEY")

# MySQL database variables
db_user = os.getenv("AZURE_USERNAME")
db_pwd = os.getenv("AZURE_PWD")
db_host = os.getenv("AZURE_URL")
db_port = os.getenv("AZURE_PORT")
db_database = os.getenv("AZURE_DB")

# print(api_key)

## URLs to Access APIs


In [22]:
TRAVEL_TIMES_URL = "http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}"

TRAFFIC_ALERTS_URL = "http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}"

WEATHER_INFORMATION_URL = "http://wsdot.wa.gov/Traffic/api/WeatherInformation/WeatherInformationREST.svc/GetCurrentWeatherInformationAsJson?AccessCode={ACCESSCODE}"

### Custom function to get API data


In [23]:
def get_api_data(url, access_key):
    """_summary_

    Args:
        url (_type_): _description_
        access_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    # create the url with the access key
    url_api = url.format(ACCESSCODE=access_key)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    # return the dataframe and the status code
    return df, response.status_code

In [24]:
def convert_dict_to_json(dataframe):

    for col in dataframe.columns:
        if dataframe[col].apply(lambda x: isinstance(x, dict)).all():
            dataframe[col] = dataframe[col].apply(lambda x: json.dumps(x))

In [25]:
def get_updates(
    travel_times_url, traffic_alerts_url, weather_information_url, access_key
):

    # Get current time
    timestamp = pd.Timestamp.now()
    df_tt, response_tt = get_api_data(travel_times_url, access_key)
    df_ta, response_ta = get_api_data(traffic_alerts_url, access_key)
    df_wa, response_wa = get_api_data(weather_information_url, access_key)

    # add pull_date_time
    df_tt["timestamp"] = timestamp
    df_ta["timestamp"] = timestamp
    df_wa["timestamp"] = timestamp

    # convert dict columns to json
    convert_dict_to_json(df_tt)
    convert_dict_to_json(df_ta)
    convert_dict_to_json(df_wa)

    df_api_fetch = pd.DataFrame(
        [
            {
                "timestamp": timestamp,
                "travel_times_response": response_tt,
                "traffic_alerts_response": response_ta,
                "weather_alerts_response": response_wa,
            }
        ]
    )

    return df_api_fetch, df_tt, df_ta, df_wa

# EXTRACT

## Call to get all info from API


In [26]:
df_api_fetch, df_tt, df_ta, df_wa = get_updates(
    TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.


# LOAD

## Export to CSV


In [27]:
# Set path to datasets folder
datasets = Path("../datasets-2")
df_api_fetch.to_csv(datasets / "api_fetch.csv", index=False)
df_tt.to_csv(datasets / "travel_times.csv", index=False)
df_ta.to_csv(datasets / "traffic_alerts.csv", index=False)
df_wa.to_csv(datasets / "weather_alerts.csv", index=False)

## Connection to MySQL Database


In [28]:
# Connect to database
connection_url = (
    f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
)
try:
    engine = create_engine(connection_url)
    print("Connection successful!")
except Exception as e:
    print("Connection unsuccessful! The error is: ", e)

Connection successful!


## Upload Data to MySQL


In [31]:
# Table api_fetch
df_api_fetch.to_sql("api_fetch", con=engine, if_exists="append", index=False)

# Table time_trave_raw
df_tt.to_sql("time_travel_raw", con=engine, if_exists="append", index=False)

# Table traffic_alerts_raw
df_ta.to_sql("traffic_alerts_raw", con=engine, if_exists="append", index=False)

# Table weather_alerts_raw
df_wa.to_sql("weather_alerts_raw", con=engine, if_exists="append", index=False)

print("Data saved to database.")

Data saved to database.


In [34]:
df_check = pd.read_sql("SELECT * FROM weather_alerts_raw", engine)
display(df_check)

,id,barometricpressure,latitude,longitude,precipitationininches,readingtime,relativehumidity,skycoverage,stationid,stationname,temperatureinfahrenheit,visibility,winddirection,winddirectioncardinal,windgustspeedinmph,windspeedinmph,timestamp
0,310,950.0,47.4748,-122.270,None,/Date(1738713607000-0800)/,99.0,N/A,1909,S 144th St on SB I-5 at mp 155.32,42.44,1,None,N/A,4.0,1.0,2025-02-04 16:23:32
1,311,NaN,47.7606,-122.184,None,/Date(1738713616000-0800)/,58.0,N/A,1910,NE 195th on SB I-405 at mp 24.58,40.82,1,5,N,0.0,0.0,2025-02-04 16:23:32
2,312,971.2,47.5090,-121.885,None,/Date(1738713608000-0800)/,88.0,N/A,1928,EB I-90 / SR-18 (Echo Lake) at mp 26.30,33.80,1,249,WSW,2.0,1.0,2025-02-04 16:23:32
3,313,993.1,47.7260,-122.324,None,/Date(1738713604000-0800)/,72.0,N/A,1966,NE 130th Street on I-5 at mp 173.75,38.30,1,None,N/A,5.0,4.0,2025-02-04 16:23:32
4,314,NaN,46.4360,-117.350,None,/Date(1738714053000-0800)/,96.0,N/A,1968,Alpowa Summit on US 12 at mp 413.36,27.68,0,3,N,3.0,2.0,2025-02-04 16:23:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,408,976.1,46.5714,-118.989,None,/Date(1738714171000-0800)/,71.0,N/A,5673,Mesa/SR17 Interchange,33.08,1,19,NNE,7.0,4.0,2025-02-04 16:23:32
99,409,NaN,47.2310,-122.457,None,/Date(1738714407000-0800)/,61.0,N/A,5678,I5 SB at SR 16 I/C Flyover,39.74,1,2,N,9.0,6.0,2025-02-04 16:23:32
100,410,NaN,47.2313,-122.461,None,/Date(1738714431000-0800)/,NaN,N/A,5680,SR 16 at I5 I/C HOV,NaN,None,None,N/A,NaN,NaN,2025-02-04 16:23:32
101,411,990.7,46.0546,-118.668,None,/Date(1738714079000-0800)/,94.0,N/A,5883,Touchet Road Interchange,30.92,1,172,S,2.0,1.0,2025-02-04 16:23:32


In [19]:
df_schema = pd.read_sql("DESCRIBE weather_alerts_raw", engine)
display(df_schema)

,Field,Type,Null,Key,Default,Extra
0,id,int unsigned,NO,PRI,None,auto_increment
1,barometricpressure,float,YES,,None,
2,latitude,float,YES,,None,
3,longitude,float,YES,,None,
4,precipitationininches,float,YES,,None,
5,readingtime,varchar(45),YES,,None,
6,relativehumidity,float,YES,,None,
7,skycoverage,varchar(45),YES,,None,
8,stationid,varchar(45),YES,,None,
9,stationname,longtext,YES,,None,


In [44]:
cols = df_wa.columns
for col in cols:
    print(col)

BarometricPressure
Latitude
Longitude
PrecipitationInInches
ReadingTime
RelativeHumidity
SkyCoverage
StationID
StationName
TemperatureInFahrenheit
Visibility
WindDirection
WindDirectionCardinal
WindGustSpeedInMPH
WindSpeedInMPH
timestamp
wa_id
